In [1]:
!git clone https://github.com/tkipf/pygcn

Cloning into 'pygcn'...
remote: Enumerating objects: 78, done.
remote: Total 78 (delta 0), reused 0 (delta 0), pack-reused 78
Unpacking objects: 100% (78/78), done.


# 新段落

In [2]:
cd /content/pygcn/pygcn

/content/pygcn/pygcn


In [3]:
!pwd

/content/pygcn/pygcn




*   修改train.py 代码from pygcn.utils import load_data, accuracy 为 from utils import load_data, accuracy
*   修改train.py 代码from pygcn.models import GCN 为 from models import GCN
*   修改models.py 代码 from pygcn.layers import GraphConvolution 为from layers import GraphConvolution




In [4]:
# 上传由!Rscript data_preprocess.R生成的input文件夹，现将其压缩为input.tgz 文件
!tar -xvf input.tgz

./._input
input/
input/._Label1.csv
input/Label1.csv
input/._inter_graph.csv
input/inter_graph.csv
input/._Label2.csv
input/Label2.csv
input/._Data2.csv
input/Data2.csv
input/._Data1.csv
input/Data1.csv
input/._intra_graph.csv
input/intra_graph.csv
input/._.ipynb_checkpoints
input/.ipynb_checkpoints/
input/._datasets.dat
input/datasets.dat


In [5]:
# 测试修改后的代码是否正常运行
!python train.py

Loading cora dataset...
Epoch: 0001 loss_train: 1.9418 acc_train: 0.1500 loss_val: 1.9129 acc_val: 0.3500 time: 0.4096s
Epoch: 0002 loss_train: 1.9260 acc_train: 0.2857 loss_val: 1.9014 acc_val: 0.3500 time: 0.0137s
Epoch: 0003 loss_train: 1.9181 acc_train: 0.3000 loss_val: 1.8900 acc_val: 0.3500 time: 0.0128s
Epoch: 0004 loss_train: 1.9014 acc_train: 0.2929 loss_val: 1.8786 acc_val: 0.3500 time: 0.0137s
Epoch: 0005 loss_train: 1.8925 acc_train: 0.2929 loss_val: 1.8672 acc_val: 0.3500 time: 0.0211s
Epoch: 0006 loss_train: 1.8762 acc_train: 0.2929 loss_val: 1.8560 acc_val: 0.3500 time: 0.0157s
Epoch: 0007 loss_train: 1.8709 acc_train: 0.2929 loss_val: 1.8451 acc_val: 0.3500 time: 0.0132s
Epoch: 0008 loss_train: 1.8604 acc_train: 0.2929 loss_val: 1.8347 acc_val: 0.3500 time: 0.0097s
Epoch: 0009 loss_train: 1.8415 acc_train: 0.2929 loss_val: 1.8248 acc_val: 0.3500 time: 0.0106s
Epoch: 0010 loss_train: 1.8284 acc_train: 0.2929 loss_val: 1.8153 acc_val: 0.3500 time: 0.0088s
Epoch: 0011 loss

*   上传data，data删除from graph
*   上传utils2
*   注释utils2的81，82行，将features转成稀疏矩阵即可，修改为
*   #features = sp.vstack((sp.vstack((datas_tr, datas_va)), datas_te)).tolil()
*   #features = preprocess_features(features)
*   修改为features = sp.vstack((sp.vstack((datas_tr, datas_va)), datas_te))



In [6]:
# Load data
from utils2 import *
adj, features, labels_binary_train, labels_binary_val, labels_binary_test, train_mask, pred_mask, val_mask, test_mask, new_label, true_label, index_guide = load_data(
    datadir='input',rgraph=True)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
load data succesfully....
assign input coordinatly....


In [7]:
features

<9105x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 3523159 stored elements in Compressed Sparse Row format>

In [8]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [9]:
features = normalize(features)

In [10]:
import torch
features = torch.FloatTensor(np.array(features.todense()))

In [11]:
features

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0015, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0025, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0036, 0.0000,  ..., 0.0000, 0.0000, 0.0004],
        [0.0000, 0.0030, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0005, 0.0010, 0.0000,  ..., 0.0000, 0.0000, 0.0005]])

In [12]:
true_label

,0
0,alpha
1,alpha
2,alpha
3,alpha
4,alpha
...,...
9100,quiescent_stellate
9101,quiescent_stellate
9102,quiescent_stellate
9103,quiescent_stellate


In [13]:
index_guide

array([ 137, 1582, 1392, ...,  108,  480,  382])

In [14]:
labels_binary_train[0]

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
import numpy as np
import scipy.sparse as sp
import torch

In [16]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [17]:
labels = encode_onehot(true_label[0])

In [18]:
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int32)

In [19]:
labels[0]

array([0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [20]:
len(labels)

9105

In [21]:
#features = torch.FloatTensor(np.array(features.todense()))
#返回值为1的索引位置
#np.where()[0] 表示行的索引，
#np.where()[1] 则表示列的索引
labels = torch.LongTensor(np.where(labels)[1])
#adj = sparse_mx_to_torch_sparse_tensor(adj)

In [22]:
labels[9102]

tensor(6)

In [23]:
idx_train = range(1483)
idx_val = range(8747, 8917)
idx_test = range(8917, 9105)
idx_pred= range(1483, 8747)

In [24]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from utils import load_data, accuracy
from models import GCN

In [25]:
# Set random seed
seed = 123
np.random.seed(seed)

In [26]:
adj, features, labels, idx_train, idx_val, idx_test = adj, features, labels, idx_train, idx_val, idx_test

In [27]:
adj

<9105x9105 sparse matrix of type '<class 'numpy.longlong'>'
	with 24063 stored elements in Compressed Sparse Row format>

In [28]:
labels

tensor([4, 4, 4,  ..., 6, 6, 6])

In [29]:
idx_train

range(0, 1483)

In [30]:
# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

_StoreAction(option_strings=['--dropout'], dest='dropout', nargs=None, const=None, default=0.5, type=<class 'float'>, choices=None, help='Dropout rate (1 - keep probability).', metavar=None)

In [31]:
# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)
optimizer = optim.Adam(model.parameters(),
                       lr=0.01, weight_decay=5e-4)

In [32]:
adj

<9105x9105 sparse matrix of type '<class 'numpy.longlong'>'
	with 24063 stored elements in Compressed Sparse Row format>

In [33]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [34]:
adj = sparse_mx_to_torch_sparse_tensor(adj)

In [35]:
adj

tensor(indices=tensor([[   0,    1,    2,  ..., 3057, 4554, 9104],
                       [   0,    1,    2,  ..., 9104, 9104, 9104]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(9105, 9105), nnz=24063, layout=torch.sparse_coo)

In [36]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [37]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [38]:
# Train model
t_total = time.time()
for epoch in range(200):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 0001 loss_train: 2.3538 acc_train: 0.0883 loss_val: 2.3628 acc_val: 0.0706 time: 0.1363s
Epoch: 0002 loss_train: 2.2643 acc_train: 0.1126 loss_val: 2.2791 acc_val: 0.1235 time: 0.0434s
Epoch: 0003 loss_train: 2.1691 acc_train: 0.1436 loss_val: 2.1694 acc_val: 0.1294 time: 0.0406s
Epoch: 0004 loss_train: 2.0856 acc_train: 0.1740 loss_val: 2.0785 acc_val: 0.1824 time: 0.0419s
Epoch: 0005 loss_train: 2.0067 acc_train: 0.1854 loss_val: 1.9906 acc_val: 0.1882 time: 0.0457s
Epoch: 0006 loss_train: 1.9394 acc_train: 0.2293 loss_val: 1.9437 acc_val: 0.2235 time: 0.0438s
Epoch: 0007 loss_train: 1.8641 acc_train: 0.2994 loss_val: 1.8819 acc_val: 0.3235 time: 0.0406s
Epoch: 0008 loss_train: 1.7899 acc_train: 0.3796 loss_val: 1.8075 acc_val: 0.3235 time: 0.0423s
Epoch: 0009 loss_train: 1.7129 acc_train: 0.4174 loss_val: 1.7323 acc_val: 0.4294 time: 0.0414s
Epoch: 0010 loss_train: 1.6406 acc_train: 0.4659 loss_val: 1.6707 acc_val: 0.4529 time: 0.0422s
Epoch: 0011 loss_train: 1.5732 acc_train

In [39]:
cd /content

/content


In [40]:
!tar -zcvf pygcn.tgz pygcn

pygcn/
pygcn/README.md
pygcn/LICENCE
pygcn/setup.py
pygcn/pygcn/
pygcn/pygcn/__pycache__/
pygcn/pygcn/__pycache__/utils.cpython-37.pyc
pygcn/pygcn/__pycache__/utils2.cpython-37.pyc
pygcn/pygcn/__pycache__/layers.cpython-37.pyc
pygcn/pygcn/__pycache__/models.cpython-37.pyc
pygcn/pygcn/__pycache__/data.cpython-37.pyc
pygcn/pygcn/utils.py
pygcn/pygcn/utils2.py
pygcn/pygcn/data.py
pygcn/pygcn/._input
pygcn/pygcn/__init__.py
pygcn/pygcn/models.py
pygcn/pygcn/train.py
pygcn/pygcn/input.tgz
pygcn/pygcn/input/
pygcn/pygcn/input/._inter_graph.csv
pygcn/pygcn/input/._Label2.csv
pygcn/pygcn/input/._Data1.csv
pygcn/pygcn/input/inter_graph.csv
pygcn/pygcn/input/Label1.csv
pygcn/pygcn/input/._.ipynb_checkpoints
pygcn/pygcn/input/._Label1.csv
pygcn/pygcn/input/._intra_graph.csv
pygcn/pygcn/input/Label2.csv
pygcn/pygcn/input/._Data2.csv
pygcn/pygcn/input/Data2.csv
pygcn/pygcn/input/datasets.dat
pygcn/pygcn/input/Data1.csv
pygcn/pygcn/input/intra_graph.csv
pygcn/pygcn/input/.ipynb_checkpoints/
pygcn/py